In [2]:
using SeisNoise, SeisIO, Plots
using Dates 
using Plots
using SeisDvv
using CSV
using Statistics
using LinearAlgebra
using Deconvolution

In [ ]:
# Define the ncf_denoise function
function ncf_denoise(img_to_denoise::Matrix{Float32}, mdate::Int64, ntau::Int64, nsv::Int64, nsv_to_rm::Int64, use_wiener::Bool, noise_power::Float64)
    m, n = size(img_to_denoise)
    nsv = min(nsv, m, n)

    U, s, V = svd(img_to_denoise)
    Xwiener = zeros(Float32, size(img_to_denoise))  # Ensure the type is Float32

    for kk in (nsv_to_rm + 1):nsv
        SV = Diagonal([kk == i ? s[kk] : 0 for i in 1:min(m, n)])

        X = U * SV * V'

        if use_wiener
            # Convert X to Float64 if necessary
            X64 = convert(Matrix{Float64}, X)
            Xwiener += wiener(X64, X64, noise_power)
        else
            Xwiener += X
        end
    end

    if use_wiener
        # Final Wiener filter application, converting Xwiener to Float64
        Xwiener64 = convert(Matrix{Float64}, Xwiener)
        return wiener(Xwiener64, Xwiener64, noise_power)
    else
        return Xwiener
    end
end

# Convert UNIX timestamp to DateTime
unixtimestamp_to_datetime(ts) = unix2datetime(ts)

function find_indices_in_window(start_times, window_start, window_end)
    return findall(t -> window_start <= unixtimestamp_to_datetime(t) <= window_end, start_times)
end

function stack_moving_window(corr_data, window_size_days, overlap_days)
    window_size = Day(window_size_days)
    overlap = Day(overlap_days)

    # Initialize a matrix for the stacked correlations and a vector for the start times
    stacked_correlations_matrix = Array{Float32}(undef, 8001, 0)
    stacked_start_times = Float64[]

    current_start_date = unixtimestamp_to_datetime(first(corr_data.t))

    while current_start_date <= unixtimestamp_to_datetime(last(corr_data.t))
        current_end_date = current_start_date + window_size - Day(1)
        indices = find_indices_in_window(corr_data.t, current_start_date, current_end_date)

        if length(indices) > 0
            window_data = deepcopy(corr_data)
            window_data.corr = corr_data.corr[:, indices]
            stacked_corr = stack(window_data, allstack=true).corr

            # Append the stacked correlation as a new column
            stacked_correlations_matrix = hcat(stacked_correlations_matrix, stacked_corr)

            # Append the start time of the window
            push!(stacked_start_times, datetime2unix(current_start_date))
        end

        current_start_date += overlap
    end

    # Create a new CorrData instance with the stacked data
    new_corr_data = deepcopy(corr_data)
    new_corr_data.corr = stacked_correlations_matrix
    new_corr_data.t = stacked_start_times

    return new_corr_data
end


# Averaging function for causal and acausal parts
function average_causal_acausal(corr, time)
    averaged_corr = copy(corr)
    causal_index = findfirst(time .>= 0)
    acausal_index = findlast(time .< 0)

    for k in 1:causal_index-1
        if acausal_index - k + 1 > 0
            averaged_corr[causal_index + k - 1] = (averaged_corr[acausal_index - k + 1] + averaged_corr[causal_index + k - 1]) / 2
        end
    end

    return averaged_corr
end

# Function to create an evenly spaced array
function evenly_spaced(a, b, n)
    h = (b - a) / (n - 1)
    collect(a:h:b)
end

function update_progress_bar(current_step::Int, start_step::Int, end_step::Int)
    total_steps = end_step - start_step
    progress = (current_step - start_step) / total_steps
    filled_length = round(Int, 50 * progress)
    bar = repeat('█', filled_length) * repeat(' ', 50 - filled_length)
    print("\rProgress: |$bar| $(round(progress * 100))% Complete")
end

function ncf_denoise(img_to_denoise::Matrix{Float32}, mdate::Int64, ntau::Int64, nsv::Int64, nsv_to_rm::Int64, use_wiener::Bool, noise_power::Float64)
    m, n = size(img_to_denoise)
    nsv = min(nsv, m, n)  # Ensure nsv does not exceed the dimensions of the image

    U, s, V = svd(img_to_denoise)  # Perform singular value decomposition
    Xwiener = zeros(Float32, m, n)  # Initialize the result matrix of type Float32
    
    total_steps = nsv - nsv_to_rm  # Calculate total steps required for the loop
    println("Starting denoising...")  # Initial message before processing starts

    for kk in (nsv_to_rm + 1):nsv
        # Perform rank-1 updates using only significant singular values
        Xwiener .+= U[:, kk] * s[kk] * V[:, kk]'  # Scale outer product by the singular value
        
        # Progress bar update
        progress = (kk - nsv_to_rm) / total_steps  # Calculate progress ratio
        filled_length = round(Int, 50 * progress)  # Calculate number of blocks to fill
        bar = repeat('█', filled_length) * repeat(' ', 50 - filled_length)  # Create progress bar
        print("\rProgress: |$bar| $(round(progress * 100))% Complete")  # Print progress bar
    end
    
    println("\nDenoising completed.")  # End message after processing completes

    return Xwiener  # Return the reconstructed, denoised image
end

In [ ]:
dd = load_corr("/data/wsd02/maleen_data/C_test/NV.NC89..HHZ.NV.NC89..HHE.jld2", "ZE")
dd = load_corr("/data/wsd02/maleen_data/C_test/NV.NC89..HHZ.NV.NC89..HHE.jld2", "ZE")

In [ ]:
ndaystack = 1
inter = Day(ndaystack)
d=stack(dd,interval=inter)